In [ ]:
# %% [markdown]
# ## Train CNN Model
# Train the standalone CNN model with cross-entropy loss.

# %% [code]
import torch
import torch.nn as nn

def train_cnn(model, train_loader, val_loader, device, optimizer, epochs=50):
    """Train the CNN model with cross-entropy loss.
    
    Args:
        model: CNNModel instance.
        train_loader: DataLoader for training data.
        val_loader: DataLoader for validation data.
        device: Device to run training on (cuda or cpu).
        optimizer: Optimizer for training.
        epochs: Number of training epochs.
    
    Returns:
        model: Trained model.
    """
    num_classes = len(train_loader.dataset.label_map)  # No blank token for cross-entropy
    print(f"Number of classes: {num_classes}")  # Debug: Should be 1400
    criterion = nn.CrossEntropyLoss()
    model.train()
    
    for epoch in range(epochs):
        total_loss = 0
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            print(f"Epoch {epoch+1}, Batch {batch_idx}, Image shape: {images.shape}, Label shape: {labels.shape}")  # Debug
            optimizer.zero_grad()
            outputs = model(images)  # Shape: (batch_size, num_classes)
            print(f"Model output shape: {outputs.shape}")  # Debug
            if outputs.size(1) != num_classes:
                raise ValueError(f"Model output has {outputs.size(1)} classes, expected {num_classes}")
            try:
                loss = criterion(outputs, labels)  # Cross-entropy: outputs (batch_size, num_classes), labels (batch_size,)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            except Exception as e:
                print(f"Loss Error: {e}")
                print(f"Output shape: {outputs.shape}, Labels shape: {labels.shape}")
                print(f"Labels sample: {labels[:5]}")  # Debug
                raise
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                try:
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
                except Exception as e:
                    print(f"Validation Loss Error: {e}")
                    print(f"Output shape: {outputs.shape}, Labels shape: {labels.shape}")
                    raise
        print(f"Validation Loss: {val_loss / len(val_loader)}")
        model.train()
    
    return model